<a href="https://colab.research.google.com/github/banggeunho/nlp_exercise/blob/master/RNN_2022_08_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 단어 단위 RNN 텍스트 생성 - 임베딩 사용

이번 챕터에서는 파이토치(PyTorch)로 인공 신경망을 이용하여 태깅 작업을 하는 모델을 만듭니다. 개체명 인식기와 품사 태거를 만드는데, 이러한 두 작업의 공통점은 RNN의 다-대-다(Many-to-Many) 작업이면서 또한 앞, 뒤 시점의 입력을 모두 참고하는 양방향 RNN(Bidirectional RNN)을 사용한다는 점입니다.

실습 챕터를 진행하기 전에 전체적으로 실습이 어떻게 진행되는지 정리해보도록 하겠습니다. 텍스트 분류 개요 챕터와 겹치는 부분에 대해서는 요약하여 설명하므로, 이해가 되지 않는 부분이 있다면 해당 챕터를 참고바랍니다.

# 양방향 RNN을 이용한 품사 태깅
파이토치(PyTorch)를 사용하여 시퀀스 레이블링의 대표적인 태스크인 품사 태깅(PoS tagging) 작업을 구현해보겠습니다.

In [6]:
# CUDA 11.1
!pip3 install torch torchvision torchaudio torchtext==0.9.0 --extra-index-url https://download.pytorch.org/whl/cu111

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu111
     |████████████████████████████████| 7.1 MB 13.0 MB/s 
     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:14:00tcmalloc: large alloc 1147494400 bytes == 0x3a074000 @  0x7f7879c57615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 1.2 MB/s eta 0:12:49tcmalloc: large alloc 1434370048 bytes == 0x7e6ca000 @  0x7f7879c57615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext.legacy as torchtext
from torchtext import data
from torchtext import datasets
import time
import random

In [2]:
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [40]:
print(device)

cuda


In [8]:
# 3개의 필드 정의
TEXT = torchtext.data.Field(lower = True)
UD_TAGS = torchtext.data.Field(unk_token = None)
PTB_TAGS = torchtext.data.Field(unk_token = None)

fields = (("text", TEXT), ("udtags", UD_TAGS), ("ptbtags", PTB_TAGS))

In [10]:
train_data, valid_data, test_data = torchtext.datasets.UDPOS.splits(fields)

downloading en-ud-v2.zip


en-ud-v2.zip: 100%|██████████| 688k/688k [00:01<00:00, 613kB/s]


extracting


In [11]:
print(f"훈련 샘플의 개수 : {len(train_data)}")
print(f"검증 샘플의 개수 : {len(valid_data)}")
print(f"테스트 샘플의 개수 : {len(test_data)}")

훈련 샘플의 개수 : 12543
검증 샘플의 개수 : 2002
테스트 샘플의 개수 : 2077


In [12]:
# 훈련 데이터의 3개의 필드 확인
print(train_data.fields)

{'text': <torchtext.legacy.data.field.Field object at 0x7fa5a5d1aa90>, 'udtags': <torchtext.legacy.data.field.Field object at 0x7fa5a5caf190>, 'ptbtags': <torchtext.legacy.data.field.Field object at 0x7fa5a5caf1d0>}


In [13]:
# 첫번째 훈련 샘플의 text 필드
print(vars(train_data.examples[0])['text'])
# 첫번째 훈련 샘플의 udtags 필드
print(vars(train_data.examples[0])['udtags'])
# 첫번째 훈련 샘플의 ptbdtags 필드
print(vars(train_data.examples[0])['ptbtags'])

['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.']
['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']
['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']


In [14]:
# 최소 허용 빈도
MIN_FREQ = 5

# 사전 훈련된 워드 임베딩 GloVe 다운로드
TEXT.build_vocab(train_data, min_freq = MIN_FREQ, vectors = "glove.6B.100d")
UD_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.34MB/s]                           
100%|█████████▉| 399999/400000 [00:15<00:00, 26586.49it/s]


In [15]:
# 상위 빈도수 20개 단어
print(TEXT.vocab.freqs.most_common(20))
# 상위 정수 인덱스 단어 10개 출력
print(TEXT.vocab.itos[:10])
# 상위 빈도순으로 udtags 출력
print(UD_TAGS.vocab.freqs.most_common())

[('the', 9076), ('.', 8640), (',', 7021), ('to', 5137), ('and', 5002), ('a', 3782), ('of', 3622), ('i', 3379), ('in', 3112), ('is', 2239), ('you', 2156), ('that', 2036), ('it', 1850), ('for', 1842), ('-', 1426), ('have', 1359), ('"', 1296), ('on', 1273), ('was', 1244), ('with', 1216)]
['<unk>', '<pad>', 'the', '.', ',', 'to', 'and', 'a', 'of', 'i']
[('NOUN', 34781), ('PUNCT', 23679), ('VERB', 23081), ('PRON', 18577), ('ADP', 17638), ('DET', 16285), ('PROPN', 12946), ('ADJ', 12477), ('AUX', 12343), ('ADV', 10548), ('CCONJ', 6707), ('PART', 5567), ('NUM', 3999), ('SCONJ', 3843), ('X', 847), ('INTJ', 688), ('SYM', 599)]


In [16]:
def tag_percentage(tag_counts): # 태그 레이블의 분포를 확인하는 함수
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]

    return tag_counts_percentages
  
print("Tag  Occurences Percentage\n")
for tag, count, percent in tag_percentage(UD_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag  Occurences Percentage

NOUN	34781	17.0%
PUNCT	23679	11.6%
VERB	23081	11.3%
PRON	18577	 9.1%
ADP	17638	 8.6%
DET	16285	 8.0%
PROPN	12946	 6.3%
ADJ	12477	 6.1%
AUX	12343	 6.0%
ADV	10548	 5.2%
CCONJ	6707	 3.3%
PART	5567	 2.7%
NUM	3999	 2.0%
SCONJ	3843	 1.9%
X	847	 0.4%
INTJ	688	 0.3%
SYM	599	 0.3%


In [18]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [19]:
# 이번 모델에서는 batch_first=True를 사용하지 않으므로 배치 차원이 맨 앞이 아님.
class RNNPOSTagger(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout): 
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = bidirectional)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)        
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))

        # embedded = [sent len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedded)

        # output = [sent len, batch size, hid dim * n directions]
        # hidden/cell = [n layers * n directions, batch size, hid dim]
        predictions = self.fc(self.dropout(outputs))

        # predictions = [sent len, batch size, output dim]
        return predictions

In [20]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = RNNPOSTagger(INPUT_DIM, 
                     EMBEDDING_DIM, 
                     HIDDEN_DIM, 
                     OUTPUT_DIM, 
                     N_LAYERS, 
                     BIDIRECTIONAL, 
                     DROPOUT)

In [21]:
# 파라미터 개수 출력
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,027,510 trainable parameters


In [23]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

model.embedding.weight.data.copy_(pretrained_embeddings)

torch.Size([3921, 100])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1020,  0.7700,  0.1169,  ..., -0.1416, -0.1932, -0.4225],
        [-0.0263,  0.0179, -0.5016,  ..., -0.8688,  0.9409, -0.2882],
        [ 0.1519,  0.4712,  0.0895,  ..., -0.4702, -0.3127,  0.1078]])

In [25]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
print(UNK_IDX)
print(PAD_IDX)

0
1


In [26]:
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM) # 0번 임베딩 벡터에는 0값을 채운다.
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM) # 1번 임베딩 벡터에는 1값을 채운다.
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1020,  0.7700,  0.1169,  ..., -0.1416, -0.1932, -0.4225],
        [-0.0263,  0.0179, -0.5016,  ..., -0.8688,  0.9409, -0.2882],
        [ 0.1519,  0.4712,  0.0895,  ..., -0.4702, -0.3127,  0.1078]])


In [27]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]
print(TAG_PAD_IDX)

0


In [41]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
model = model.to(device)
criterion = criterion.to(device)
batch = next(iter(train_iterator))
prediction = model(batch.text)

In [32]:
print(prediction.shape)
prediction = prediction.view(-1, prediction.shape[-1])
print(prediction.shape)
print(batch.udtags.shape)
print(batch.udtags.view(-1).shape)

torch.Size([2944, 18])
torch.Size([2944, 18])
torch.Size([46, 64])
torch.Size([2944])


In [52]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    미니 배치에 대한 정확도 출력
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(device)

In [37]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        text = batch.text.to(device)
        tags = batch.udtags.to(device)

        
        optimizer.zero_grad()

        #text = [sent len, batch size]     
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        predictions = predictions.view(-1, predictions.shape[-1]) # #predictions = [sent len * batch size, output dim]
        tags = tags.view(-1) # tags = [sent len * batch_size]

        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        loss = criterion(predictions, tags)

        acc = categorical_accuracy(predictions, tags, tag_pad_idx)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [38]:
def evaluate(model, iterator, criterion, tag_pad_idx):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text = batch.text.to(device)
            tags = batch.udtags.to(device)

            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            loss = criterion(predictions, tags)

            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [53]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01
	Train Loss: 0.435 | Train Acc: 86.37%
	 Val. Loss: 0.558 |  Val. Acc: 82.97%
Epoch: 02
	Train Loss: 0.318 | Train Acc: 89.79%
	 Val. Loss: 0.496 |  Val. Acc: 84.13%
Epoch: 03
	Train Loss: 0.274 | Train Acc: 91.10%
	 Val. Loss: 0.468 |  Val. Acc: 85.00%
Epoch: 04
	Train Loss: 0.245 | Train Acc: 92.00%
	 Val. Loss: 0.443 |  Val. Acc: 85.58%
Epoch: 05
	Train Loss: 0.225 | Train Acc: 92.60%
	 Val. Loss: 0.434 |  Val. Acc: 85.79%
Epoch: 06
	Train Loss: 0.208 | Train Acc: 93.16%
	 Val. Loss: 0.432 |  Val. Acc: 86.08%
Epoch: 07
	Train Loss: 0.195 | Train Acc: 93.52%
	 Val. Loss: 0.415 |  Val. Acc: 86.37%
Epoch: 08
	Train Loss: 0.183 | Train Acc: 93.93%
	 Val. Loss: 0.415 |  Val. Acc: 86.32%
Epoch: 09
	Train Loss: 0.172 | Train Acc: 94.23%
	 Val. Loss: 0.412 |  Val. Acc: 86.56%
Epoch: 10
	Train Loss: 0.161 | Train Acc: 94.65%
	 Val. Loss: 0.405 |  Val. Acc: 86.93%


In [54]:
test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.411 |  Test Acc: 87.19%
